In [38]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
%pip install geopy
from geopy.geocoders import Nominatim
import urllib.request
import json
%pip install bs4
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Libraries imported.


In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [4]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [7]:
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [8]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Borough'], Brooklyn_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [9]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [10]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [11]:
#https://developer.foursquare.com/docs/resources/categories
#Sushi = 4bf58dd8d48988d1d2941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
newyork_venues_sushi = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
newyork_venues_sushi.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Inaka,40.625141,-74.030418,Sushi Restaurant
1,Bay Ridge,40.625801,-74.030621,Chopstix Restaurant,40.622670,-74.025465,Asian Restaurant
2,Bay Ridge,40.625801,-74.030621,Arirang Hibachi Steakhouse,40.621168,-74.029299,Japanese Restaurant
3,Bay Ridge,40.625801,-74.030621,Sakana Sushi & Asian Bistro,40.623623,-74.024910,Sushi Restaurant
4,Bay Ridge,40.625801,-74.030621,Sushi Yu III,40.632485,-74.027321,Sushi Restaurant


In [15]:
newyork_venues_sushi.shape

(320, 7)

In [12]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [13]:
map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_sushi, 'red', map_newyork_sushi)

map_newyork_sushi

In [19]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [20]:
brooklyn_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
brooklyn_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,10,10,10,10,10,10
Bay Ridge,11,11,11,11,11,11
Bensonhurst,8,8,8,8,8,8
Bergen Beach,3,3,3,3,3,3
Boerum Hill,9,9,9,9,9,9
Borough Park,1,1,1,1,1,1
Brighton Beach,7,7,7,7,7,7
Brooklyn Heights,10,10,10,10,10,10
Bushwick,4,4,4,4,4,4



3. Analyze Each Neighborhood

In [23]:

# one hot encoding
brooklyn_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Neighborhood,Asian Restaurant,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Fish Market,Italian Restaurant,Japanese Restaurant,Kosher Restaurant,Poke Place,Snack Place,Sushi Restaurant,Thai Restaurant
0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Bay Ridge,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Bay Ridge,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,1,0


In [24]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Asian Restaurant,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Fish Market,Italian Restaurant,Japanese Restaurant,Kosher Restaurant,Poke Place,Snack Place,Sushi Restaurant,Thai Restaurant
0,Bath Beach,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.100000,0.0,0.000000,0.000000,0.900000,0.000
1,Bay Ridge,0.090909,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.181818,0.0,0.000000,0.000000,0.727273,0.000
2,Bensonhurst,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.875000,0.125
3,Bergen Beach,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.000000,0.000000,0.666667,0.000
4,Boerum Hill,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000
5,Borough Park,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000
6,Brighton Beach,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000
7,Brooklyn Heights,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.000000,0.000000,0.800000,0.000
8,Bushwick,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.250000,0.0,0.000000,0.000000,0.750000,0.000
9,Carroll Gardens,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.142857,0.857143,0.000


In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Sushi Restaurant,Japanese Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant,Café
1,Bay Ridge,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant
2,Bensonhurst,Sushi Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Japanese Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant,Café
3,Bergen Beach,Sushi Restaurant,Japanese Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant,Café
4,Boerum Hill,Sushi Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Japanese Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant,Café


Cluster Neighborhoods

In [41]:
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 1, 2, 4, 2, 1, 1, 2], dtype=int32)

In [40]:
# add clustering labels


brooklyn_merged = Brooklyn_data
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant
1,Brooklyn,Bensonhurst,40.611009,-73.995180,Sushi Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Japanese Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant,Café
2,Brooklyn,Sunset Park,40.645103,-74.010316,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Japanese Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant
3,Brooklyn,Greenpoint,40.730201,-73.954241,Sushi Restaurant,Japanese Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant,Café
4,Brooklyn,Gravesend,40.595260,-73.973471,Sushi Restaurant,Thai Restaurant,Snack Place,Poke Place,Kosher Restaurant,Japanese Restaurant,Italian Restaurant,Fish Market,Chinese Restaurant,Café
